In [ ]:
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from pyspark.sql import types as t
from pyspark.sql import Window
from pyspark.sql.types import TimestampType

import datetime
from pyspark.sql.window import *

from databricks import sql
import os

In [ ]:

# LISTINGS
listings_raw = #REMOVED

# LISTING HISTORY
listingHistory_raw = #REMOVED

# OPPORTUNITY
opportunity_raw = #REMOVED

# OPPORTUNITY STAGE HISTORY
oppHistory_raw = #REMOVED
# CAMPAIGN
campaign_raw = #REMOVED

# CAMPAIGN MEMBER
campaignMember_raw = #REMOVED

# CONTACT
contact_raw = #REMOVED

# CURRENCY CONVERSION
currency_raw = #REMOVED

# COMPANY ACCOUNT
account_raw = #REMOVED

# PROPERTY
property_raw = #REMOVED

# ASSOCIATE PROPERTY
associateProperty_raw = #REMOVED

# LEAD
lead_raw = #REMOVED

# ADOBE AEM_ADW
adobe_raw = #REMOVED

# BOX ACTIVITIES
box_raw = #REMOVED

# BIDS
bids_raw = #REMOVED

# BOX CURRENT
box_current = #REMOVED

# BOX HISTORY
box_hist_df = #REMOVED

In [ ]:
#Listing History Columns
listingHistory = listingHistory_raw.select(col('Id').alias('lh_Id'), col('ParentId').alias('lh_ParentId'), col('CreatedDate').alias('lh_CreatedDate'), col('Field').alias('lh_Field'), col('OldValue').alias('lh_OldValue'), col('NewValue').alias('lh_NewValue'), col('IsDeleted').alias('lh_IsDeleted'))

#Listings Columns
listings = listings_raw.select(col('Id').alias('l_Id'), col('Name').alias('l_Name'), col('CreatedDate').alias('l_CreatedDate'), col('Display_Status__c').alias('l_Display_Status__c'), col('Asking_Price__c').alias('l_Asking_Price__c'), col('Opportunity__c').alias('l_Opportunity__c'), col('Campaign__c').alias('l_Campaign__c'), col('Display_On_Web__c').alias('l_Display_On_Web__c'), col('IsDeleted').alias('l_IsDeleted'))

#Associate Property Columns
associateProperty = associateProperty_raw.select(col('Id').alias('ap_Id'), col('Opportunity__c').alias('ap_Opportunity__c'), col('Property__c').alias('ap_Property__c'), col('IsDeleted').alias('ap_IsDeleted'))

#Property Columns
property = property_raw.select(col('Id').alias('p_Id'), col('Asset_Type__c').alias('p_Asset_Type__c'), col('Longitude__c').alias('p_Longitude__c'), col('Latitude__c').alias('p_Latitude__c'), col('State__c').alias('p_State__c'), col('City__c').alias('p_City__c'), col('IsDeleted').alias('p_IsDeleted'), col('Zip_Code__c').alias('p_Zip_Code'))

#Campaign Member Columns
campaignMember_columns = campaignMember_raw.select(col('Id').alias('cm_Id'), col('LeadId').alias('cm_LeadId'), col('ContactId').alias('cm_ContactId'), col('CampaignId').alias('cm_CampaignId'), col('Status').alias('cm_Status'), col('Executed_Agreement__c').alias('cm_Executed_Agreement__c'), col('Bid_Submitted__c').alias('cm_Bid_Submitted__c'), col('Inspection_Date__c').alias('cm_Inspection_Date__c'), col('Offering_Investment_Memorandum_Open_Date__c').alias('cm_Offering_Investment_Memorandum_Open_Date__c'), to_date(col('Tour_Date_Time__c')).alias('cm_Tour_Date'), col('IsDeleted').alias('cm_IsDeleted'), col('Reason_For_Exclusion__c').alias('cm_Reason_For_Exclusion__c'), col('Suppressed_in_Eloqua__c'), 'Bulk_Email_Opt_Out__c', 'HasOptedOutOfEmail', 'Inactive__c', 'Exclude_from_Campaign_Email__c', 'Company_Status__c')

#Opportunity Columns
opportunity = opportunity_raw.select(col('Id').alias('o_Id'), col('AccountId').alias('o_AccountId'), col('Deal_Value__c').alias('o_Deal_Value__c'), col('CurrencyIsoCode').alias('o_CurrencyIsoCode'), col('Confidential__c').alias('o_Confidential__c'), col('IsDeleted').alias('o_IsDeleted'))

#Currency Columns
currency_columns = currency_raw.select(col('Id').alias('curr_Id'), col('ConversionRate').alias('curr_ConversionRate'), col('IsoCode').alias('curr_IsoCode'))

#Account Columns
account = account_raw.select(col('Id').alias('ac_Id'), col('Name').alias('ac_Name'), col('IsDeleted').alias('ac_IsDeleted'))

#Adobe Columns
adobe = adobe_raw.select( col('Date').alias('ad_Date'), col('PropertyListingIDc49prop49').alias('ad_PropertyListingIDc49prop49'), col('CapForceIDv84evar84').alias('ad_CapForceIDv84evar84'), col('Visits').alias('ad_Visits'), col('PageViews').alias('ad_PageViews'))

#Contact Columns
contact = contact_raw.select(col('Id').alias('c_Id'), col('AccountId').alias('c_AccountId'), col('Name').alias('c_Name'), col('Full_Name__c').alias('c_Full_Name__c'), col('Email').alias('c_Email'), col('MobilePhone').alias('c_MobilePhone'), col('Phone').alias('c_Phone'), col('Box_App_User_ID__c').alias('c_Box_App_User_ID__c'), col('IsDeleted').alias('c_IsDeleted'), col('User__c').alias('c_UserId'))

#Lead Columns
lead = lead_raw.select(col('Id').alias('ld_Id'), col('Name').alias('ld_Name'), col('Company').alias('ld_Company'), col('IsDeleted').alias('ld_IsDeleted'))

#Campaign Member Columns
campaignMember = campaignMember_columns.filter((col('cm_Status').isin('CA/NDA Signed', 'Bid Submitted / Terms Received', 'Quote Submitted / Terms Received'))
                                                | (col('cm_Executed_Agreement__c').isNotNull())
                                                | (col('cm_Bid_Submitted__c').isNotNull())
                                                | (col('cm_Inspection_Date__c').isNotNull())
                                                | (col('cm_Offering_Investment_Memorandum_Open_Date__c').isNotNull()))

#Currency Columns
currency = currency_columns.filter(col('curr_IsoCode').isNotNull())

#Capforce Campaign Columns
campaignCapforce = campaign_raw.select(col('Id').alias('camp_campaignId'), col('Number_of_Campaign_Members__c').cast('int').alias('camp_No_of_CampaignMembers'), col('NumberOfContacts').cast('int').alias('camp_NumberOfContacts'), col('Opportunity__c').alias('camp_OppId'), col('Content_Status__c').alias('camp_Content_Status'), col('IsActive').alias('camp_IsActive'), col('Team__c').alias('camp_TeamId'), col('Status').alias('camp_Status'))

#Lead Broker Columns
leadBroker = opportunity_raw.select(col('Id').alias('oppId'), col('Primary_Broker__c').alias('Primary_Broker_User_Id'), col('Secondary_Broker__c').alias('Secondary_Broker_User_Id'))

In [ ]:
#Join leadBroker raw data with contact user info and pull data for primary broker
primaryBrokerInfo = leadBroker.join(contact, leadBroker.Primary_Broker_User_Id == contact.c_UserId, 'left').select('oppId', 'Primary_Broker_User_Id', col('c_Id').alias('Primary_Broker_Contact_Id'), col('c_Name').alias('Primary_Broker_Name'), col('c_Email').alias('Primary_Broker_Email'), col('c_Phone').alias('Primary_Broker_Phone'))

#Join leadBroker raw data with contact user info and pull data for secondary broker
secondaryBrokerInfo = leadBroker.join(contact, leadBroker.Secondary_Broker_User_Id == contact.c_UserId, 'left').select('oppId', 'Secondary_Broker_User_Id', col('c_Id').alias('Secondary_Broker_Contact_Id'), col('c_Name').alias('Secondary_Broker_Name'), col('c_Email').alias('Secondary_Broker_Email'), col('c_Phone').alias('Secondary_Broker_Phone'))

#Join primary broker and secondary broker together
leadBrokerInfoJoin = primaryBrokerInfo.join(secondaryBrokerInfo, ['oppId'], 'left')

#Select and aggregate relevant data into a single row per opportunity. 
leadBrokerInfo = leadBrokerInfoJoin.groupBy(col('oppId')).agg(max(col('Primary_Broker_User_Id')).alias('Primary_Broker_User_Id'), max(col('Primary_Broker_Contact_Id')).alias('Primary_Broker_Contact_Id'), max(col('Primary_Broker_Name')).alias('Primary_Broker_Name'), max(col('Primary_Broker_Email')).alias('Primary_Broker_Email'), max(col('Primary_Broker_Phone')).alias('Primary_Broker_Phone'), max(col('Secondary_Broker_User_Id')).alias('Secondary_Broker_User_Id'), max(col('Secondary_Broker_Contact_Id')).alias('Secondary_Broker_Contact_Id'), max(col('Secondary_Broker_Name')).alias('Secondary_Broker_Name'),  max(col('Secondary_Broker_Email')).alias('Secondary_Broker_Email'), max(col('Secondary_Broker_Phone')).alias('Secondary_Broker_Phone'))

In [ ]:
#IC_Listings
ltemp1 = listingHistory.filter((listingHistory.lh_Field == 'Display_on_Web__c') & (listingHistory.lh_NewValue == 'true'))  #<------ Currently live listings on Investor Center

ltemp2 = ltemp1.groupBy(ltemp1.lh_ParentId).agg(min('lh_CreatedDate').alias('Publish_Date'),
                                             datediff(current_date(), min('lh_CreatedDate').cast('date')).alias('Published_Days'))

ic_listings = ltemp2.select(col('lh_ParentId').alias('Listing_Id'), to_date(col('Publish_Date')).alias('Publish_Date'), col('Published_Days'))   #<---------- Listing created till today = No. of Published days


#Property Table
listingstemp = listings.select('l_Id', 'l_Name', col('l_CreatedDate').cast('date').alias('l_CreatedDate'), 'l_Display_Status__c', col('l_Asking_Price__c').cast('int').alias('l_Asking_Price__c'), 'l_Opportunity__c', 'l_Campaign__c', 'l_Display_On_Web__c', 'l_IsDeleted')
listingstemp = listingstemp.withColumnRenamed('l_Id', 'Listings_Id')
listingstemp = listingstemp.where(listingstemp.l_IsDeleted == 'false')

proptemp1 = listingstemp.join(associateProperty, listingstemp.l_Opportunity__c == associateProperty.ap_Opportunity__c, how = 'left').drop(associateProperty.ap_Opportunity__c)
proptemp2 = proptemp1.join(property, proptemp1.ap_Property__c == property.p_Id, how = 'left').drop(property.p_Id)

property_table = proptemp2.groupBy('Listings_Id').agg(max(col('p_Asset_Type__c')).alias('Property_Type'),
                                                      max(col('p_Longitude__c')).alias('Longitude'),
                                                      max(col('p_Latitude__c')).alias('Latitude'),
                                                      max(col('p_State__c')).alias('State'),
                                                      max(col('p_Zip_Code')).alias('Zip_Code'),
                                                      max(col('p_City__c')).alias('City'))   #<---------- Property demographics associated to the listings  


#Member Activity --- filters Campaign members who've executed CA and/or in later stages of the campaign
cmtemp1 = campaignMember.where((campaignMember.cm_Status.isin('CA/NDA Signed', 'Bid Submitted / Terms Received', 'Quote Submitted / Terms Received', 'Offer Submitted')) | campaignMember.cm_Executed_Agreement__c.isNotNull() | campaignMember.cm_Bid_Submitted__c.isNotNull())
cmtemp2 = cmtemp1.join(listings, cmtemp1.cm_CampaignId == listings.l_Campaign__c, 'inner').drop(listings.l_Campaign__c)
member_activity = cmtemp2.groupBy('cm_CampaignId', coalesce('cm_ContactId', 'cm_LeadId').alias('Contact_Id'), 'cm_LeadId').agg(max(when(cmtemp2.cm_Status == 'CA/NDA Signed', 1)
                                                                                                                                   .when(cmtemp2.cm_Executed_Agreement__c.isNotNull(), 1)
                                                                                                                                   .otherwise(0)).alias('Executed_CA'),   #<---------- Contact that signed a CA/NDA per listing
                                                                                                                               sum(when((cmtemp2.cm_Status == 'Bid Submitted / Terms Received') |
                                                                                                                                        (cmtemp2.cm_Status == 'Quote Submitted / Terms Received') |
                                                                                                                                        (cmtemp2.cm_Status == 'Offer Submitted') |
                                                                                                                                        cmtemp2.cm_Bid_Submitted__c.isNotNull(), 1)
                                                                                                                                   .otherwise(0)).alias('Offer_Submitted'),    #<---------- Offer submitted (Bids/Quotes)
                                                                                                                               max(to_date(cmtemp2.cm_Executed_Agreement__c, "yyyy-MM-dd")).alias('Date_Executed_CA'),
                                                                                                                               max(to_date(cmtemp2.cm_Bid_Submitted__c, "yyyy-MM-dd")).alias('Date_Offer_Submitted'),
                                                                                                                               max(to_date(cmtemp2.cm_Inspection_Date__c, "yyyy-MM-dd")).alias('Inspection_Date'),
                                                                                                                               max(to_date(cmtemp2.cm_Offering_Investment_Memorandum_Open_Date__c, "yyyy-MM-dd")).alias('Memo_Open_Date'),
                                                                                                                               max(cmtemp2.cm_Tour_Date).alias('Tour_Date'),
                                                                                                                               max(when(cmtemp2.cm_Reason_For_Exclusion__c.isNotNull(), 1)
                                                                                                                                   .when(cmtemp2.Suppressed_in_Eloqua__c == 'true', 1)
                                                                                                                                   .when(cmtemp2.Bulk_Email_Opt_Out__c == 'true', 1)
                                                                                                                                   .when(cmtemp2.HasOptedOutOfEmail == 'true', 1)
                                                                                                                                   .when(cmtemp2.Inactive__c == 'true', 1)
                                                                                                                                   .when(cmtemp2.Exclude_from_Campaign_Email__c == 'true', 1)
                                                                                                                                   .when(cmtemp2.Company_Status__c == 'Excluded', 1)
                                                                                                                                   .otherwise(0)).alias('Excluded'))   #<---------- Is the contact excluded from being contacted (1:Yes, 0:Not excluded)


In [ ]:
tempOutput1 = listings.join(ic_listings, ic_listings.Listing_Id == listings.l_Id, 'inner').drop(ic_listings.Listing_Id)
tempOutput2 = tempOutput1.join(member_activity, tempOutput1.l_Campaign__c == member_activity.cm_CampaignId, 'left').drop(member_activity.cm_CampaignId)
tempOutput3 = tempOutput2.join(opportunity, opportunity.o_Id == tempOutput2.l_Opportunity__c, 'left').drop(tempOutput2.l_Opportunity__c)
tempOutput4 = tempOutput3.join(currency, tempOutput3.o_CurrencyIsoCode == currency.curr_IsoCode, 'left').drop(currency.curr_IsoCode)
tempOutput5 = tempOutput4.join(account, account.ac_Id == tempOutput4.o_AccountId, 'left').drop(account.ac_Id)
capListings = tempOutput5.join(property_table, property_table.Listings_Id == tempOutput5.l_Id, 'left').drop(property_table.Listings_Id) 

#capListings is by ContactId/LeadId and Date (CA executed, offer submitted, inspection and memo date )


#Capforce Listings active only
cActiveListings = capListings.withColumn('Status', when(col('l_Display_On_Web__c') == 'true', 'Active').when(col('l_Display_On_Web__c') == 'false', 'Inactive').otherwise('Unknown')).withColumn('Private_Listing_Name', when(col('l_Display_Status__c') == 'Private', concat_ws(' ', col('Property_Type'), col('City'))).otherwise(None)).drop(capListings.l_Display_On_Web__c)#.where(col('Status') == 'Active')

cActiveListings = cActiveListings.where((cActiveListings.Status == 'Active') | (cActiveListings.l_CreatedDate >= '2022-01-01'))   #<---------- ACTIVE LISTINGS or CREATED DATE FILTERS

# Join Active listings to account
c_companies = contact.select('c_Id', 'c_AccountId')
capActiveListings = cActiveListings.join(c_companies, cActiveListings.Contact_Id == c_companies.c_Id, how = 'left').drop(c_companies.c_Id)


#Aggregate by Contacts and Companies
cap_agg1 = capActiveListings.groupby('l_Id','l_Name', 'l_CreatedDate', 'l_Display_Status__c', 'l_Asking_Price__c', 'Publish_Date', 'Published_Days', 'l_Campaign__c', 'o_Id', 'o_AccountId', 'o_Deal_Value__c',  'o_CurrencyIsoCode', 'o_Confidential__c', 'curr_ConversionRate', 'ac_Name', 'Property_Type', 'Longitude', 'Latitude', 'State', 'City', 'Zip_Code', 'Status', 'Private_Listing_Name').agg(sum('Executed_CA').alias('#ofExecutedCA'), 
                                                                                                                                                                                                                                                                                                                                                                                              sum('Offer_Submitted').alias('#ofBidsbyContactsCA'),
                                                                                                                                                                                                                                                                                                                                                                                              countDistinct('c_AccountId').alias('#ofContactCompaniesCA'),
                                                                                                                                                                                                                                                                                                                                                                                              countDistinct((when(capActiveListings.Offer_Submitted == 1, capActiveListings.c_AccountId).otherwise(lit(None)))).alias('#ofBiddingCompanies'),
                                                                                                                                                                                                                                                                                                                                                                                              sum(when(capActiveListings.Tour_Date.isNotNull(), 1).otherwise(0)).alias('#PropertyTours'),
                                                                                                                                                                                                                                                                                                                                                                                              sum('Excluded').alias('MemberExclusion'))


#Capforce Listings level info
capData = cap_agg1.select(col('l_Id').alias('ListingId'), col('l_Name').alias('ListingName'), col('l_Campaign__c').alias('CampaignId'), col('o_Id').alias('OpportunityID'), col('o_AccountId').alias('CompanyID'), col('ac_Name').alias('CompanyName'), '#ofExecutedCA',  '#ofBidsbyContactsCA', '#ofContactCompaniesCA', '#ofBiddingCompanies', '#PropertyTours', col('l_CreatedDate').cast('date').alias('ListingCreatedDate'), col('l_Display_Status__c').alias('DisplayStatus'), 'Publish_Date', 'Published_Days', col('l_Asking_Price__c').cast('double').alias('AskingPrice'), col('o_Deal_Value__c').cast('double').alias('DealValue'),  col('o_CurrencyIsoCode').alias('ISOCode'), col('curr_ConversionRate').alias('CurConvRate'), col('o_Confidential__c').alias('Confidential'), 'Property_Type', 'Longitude', 'Latitude', 'State', 'City', 'Zip_Code', 'Status', 'Private_Listing_Name', 'MemberExclusion')


In [ ]:
boxlistings = boxActivities.where(col('EventCreatedBy') != "JLL-Global-CM-GPL-CapForce")
boxlistingsagg= boxlistings.select('ListingID', 'EventCreatedByLoginID', 'EventCreatedBy', 'EventDate', 'ObjName', row_number().over(Window.partitionBy('EventCreatedByLoginID', 'ObjName').orderBy(desc('EventDate'))).alias('Row_Num'))

boxlistingsagg = boxlistingsagg.filter(boxlistingsagg.Row_Num == 1)

boxdownloadsagg = boxlistingsagg.groupBy('ListingID', 'EventCreatedByLoginID', 'EventCreatedBy').agg(max(col('EventDate').cast('date')).alias('Download_Date'),
                                                                                      count('ObjName').alias('BoxDownloads')) 

### CONTACT level box download aggregate                                                                                      
boxtemp1 = boxdownloadsagg.select('ListingID', 'EventCreatedByLoginID', col("EventCreatedBy").alias("Contact_Name"), 'Download_Date', 'BoxDownloads')
boxtemp2 = boxtemp1.join(contact, boxtemp1.EventCreatedByLoginID == contact.c_Box_App_User_ID__c, how = 'left').drop(contact.c_Box_App_User_ID__c).drop(contact.c_IsDeleted)
boxdownloads = boxtemp2.select('ListingID', col('c_ID').alias('BoxContactId'), 'Contact_Name', col('c_AccountId').alias('BoxCompanyId'), 'Download_Date','BoxDownloads')

### LISTINGS level box download aggregate
boxDownloadData = boxdownloads.groupBy('ListingID').agg(sum('BoxDownloads').alias('TotalBoxDownloads'))

In [ ]:
adobetemp1 = adobe.withColumn('ad_ListingID', regexp_replace(col('ad_PropertyListingIDc49prop49'), '-en_US', '')).drop(adobe.ad_PropertyListingIDc49prop49).withColumnRenamed('ad_CapForceIDv84evar84', 'ad_ContactId')

#ListingID and ContactId level aggregate
adobetemp2 = adobetemp1.groupby('ad_ListingID', 'ad_ContactId').agg(sum('ad_Visits').alias('WebVisits'),
                                                                    sum('ad_PageViews').alias('PageViews'))

#Listing level Aggregate
adobeData = adobetemp1.groupby('ad_ListingID').agg(sum('ad_Visits').alias('TotalVisits'),
                                                   sum('ad_PageViews').alias('PageViews'),
                                                   countDistinct('ad_ContactId').alias('UniqueVisitors'))

In [ ]:
## Adobe Daily trend chart
listingIDs = capData.select('ListingID').distinct()

#Filter only active listings in capforce
adobetemp3 = adobetemp1.join(listingIDs, adobetemp2.ad_ListingID == listingIDs.ListingID, how = 'inner').drop(listingIDs.ListingID)

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
adobetemp4 = adobetemp3.withColumn('ad_Date', date_format(to_date(adobetemp3.ad_Date, 'MMMM dd, yyyy'), 'yyyy-MM-dd'))

adobetemp5 = adobetemp4.groupby('ad_ListingID', 'ad_Date').agg(sum('ad_Visits').alias('TotalVisits'),
                                                   sum('ad_PageViews').alias('PageViews'),
                                                   countDistinct('ad_ContactId').alias('UniqueVisitors'))

spec  = Window.partitionBy('ad_ListingID')
adobetemp6 = adobetemp5.withColumn('max_Date', max(adobetemp5.ad_Date).over(spec))
adobetemp7 = adobetemp6.withColumn('min_Date', min(adobetemp5.ad_Date).over(spec))
adobetemp8 = adobetemp7.select(col('ad_ListingID'), col('ad_Date'), col('TotalVisits'), col('PageViews'), col('UniqueVisitors'))

In [ ]:
#Fill missing dates within range of min and max with data filled with 0s
tempgroupby = adobetemp7.groupBy(adobetemp7.ad_ListingID).agg(min(col('min_Date')).alias('min_Date'), max(col('max_Date')).alias('max_Date'))
tempgroupby2 = tempgroupby.select(col('ad_ListingID'), to_date(col('max_Date')).alias('max_Date'))

#Limit dates range to previous 90 days
L90days = tempgroupby2.withColumn('L90', date_add(col('max_Date'), -89))

dates_range = L90days.select(
    col('ad_ListingID').alias('ad_ListingID'), col('max_Date'),
    expr('sequence(L90, max_Date, interval 1 day)').alias('ad_Date')
)

dates_range2 = dates_range.withColumn('ad_Date', explode('ad_Date'))
dates_range3 = dates_range2.select(col('ad_ListingID'), col('ad_Date').cast('string'), col('max_Date'))

adobetemp9 = dates_range3.join(adobetemp8, on=['ad_Date', 'ad_ListingID'],how="left")
adobetemp10 = adobetemp9.fillna(0, subset=['TotalVisits', 'PageViews', 'UniqueVisitors'])

adobeOpId = listings_raw.select(col('Opportunity__c').alias('OpportunityId'), col('Id').alias('ListingId'))
adobetemp11 = adobetemp10.join(adobeOpId, adobetemp10.ad_ListingID == adobeOpId.ListingId, 'left').drop(adobeOpId.ListingId)

In [ ]:
#Missing CampaignIDs
mCampaignIds = #REMOVED

# EMAIL CAMPAIGN
eCampaign_snap = #REMOVED

#Join Eloqua Campaign table with Custom pulled missing campaign Ids data set. -----> crmId = Capforce CampaignID
eCampaign_snap = eCampaign_snap.join(mCampaignIds, eCampaign_snap.id == mCampaignIds.CAMPAIGNID, how = 'left').drop(mCampaignIds.CAMPAIGNID).drop(mCampaignIds.CRMID)
eCampaign_snap = eCampaign_snap.withColumn('crmId', when(eCampaign_snap.crmId == '', eCampaign_snap.CRMMISSING_ID).otherwise(eCampaign_snap.crmId))

# EMAIL SENDS
eSend_snap = #REMOVED

# EMAIL CONTACTS
eContacts_snap = #REMOVED

# EMAIL BOUNCEBACKS
eBounceback_snap = #REMOVED

# EMAIL CLICKTHROUGHS
eClick_snap = #REMOVED

# EMAIL OPENS
eOpen_snap = #REMOVED


###### ---------- SELECT REQUIRED COLUMNS

#Contacts
contactsData = eContacts_snap.select('contactID', col('C_SFDCContactID').alias('cap_ContactId'), col('C_SFDCLeadID').alias('cap_LeadId'), col('C_SFDCAccountID').alias('cap_AccountId'),  col('C_EmailAddress').alias('emailaddress'), col('C_FirstAndLastName').alias('C_Name'),  'C_Company', 'isSubscribed', 'isBounced')

#Sends
sendData = eSend_snap.select(date_format(to_date(col('ActivityDate')), "MM-dd-yyyy").alias('SendDate'), 'ActivityType', 'ContactId', 'CampaignId', 'CampaignName') #'EmailRecipientId', 'ExternalId', 'UserCompany', 'UserId', 'UserEmailSenderAddress','AssetId', 'AssetName', 'EmailAddress', 'Id', 'SubjectLine',  

#Campaign
campaignData = eCampaign_snap.select(col('id').alias('eloq_campaignId'), col('crmId').alias('cap_campaignId'), 'campaignCategory', 'campaignType', col('name').alias('eloq_campaignName'), from_unixtime(col('createdAt'),"MM-dd-yyyy").alias('createdAt'))

#Opens
openData = eOpen_snap.select(date_format(to_date(col('ActivityDate')), "MM-dd-yyyy").alias('OpenDate'), col('ContactId').alias('eo_ContactId'), col('Id').alias('OpenId'), col('CampaignId').alias('eo_CampaignId'))

#clickthroughs
clickData = eClick_snap.select(date_format(to_date(col('ActivityDate')), "MM-dd-yyyy").alias('ClickDate'), col('ContactId').alias('ec_ContactId'), col('Id').alias('ClickId'), col('CampaignId').alias('ec_CampaignId'))

#Bouncebacks
bounceData = eBounceback_snap.select(date_format(to_date(col('ActivityDate')), "MM-dd-yyyy").alias('BouncedDate'), col('ContactId').alias('eb_ContactId'), col('Id').alias('BounceId'), col('CampaignId').alias('eb_CampaignId'))




###### ---------- AGGREGATION

#Join Eloqua campaign and Contacts with Email Sends -- ContactIds, CampaignIds, Email Sends
tempSend1 = campaignData.join(sendData, campaignData.eloq_campaignId == sendData.CampaignId, how = 'left').drop(sendData.CampaignId)
tempSend2 = tempSend1.join(contactsData, tempSend1.ContactId == contactsData.contactID, how = 'left').drop(contactsData.contactID)

#Agg Sends
sendagg1 = tempSend2.groupBy('cap_campaignId').agg(countDistinct('eloq_campaignId').alias('EmailCampaigns'),
                                               count('cap_ContactId').alias('EmailsSent'),
                                               countDistinct('cap_ContactId').alias('#SentContacts'), 
                                               countDistinct('cap_AccountId').alias('#SentCompanies'))

#Join Eloqua campaign and Contacts with Email Opens -- ContactIds, CampaignIds, Email Opens
tempOpen1 = campaignData.join(openData, campaignData.eloq_campaignId == openData.eo_CampaignId, how = 'left').drop(openData.eo_CampaignId)
tempOpen2 = tempOpen1.join(contactsData, tempOpen1.eo_ContactId == contactsData.contactID, how = 'left').drop(contactsData.contactID)

#Agg Opens
openagg1 = tempOpen2.groupBy('cap_campaignId').agg(#countDistinct('eloq_campaignId').alias('EmailCampaigns'),
                                               count('cap_ContactId').alias('EmailsOpened'),
                                               countDistinct('cap_ContactId').alias('#OpenedContacts'), 
                                               countDistinct('cap_AccountId').alias('#OpenedCompanies'))

#Join Eloqua campaign and Contacts with Email Clicks -- ContactIds, CampaignIds, Email Clicks
tempClick1 = campaignData.join(clickData, campaignData.eloq_campaignId == clickData.ec_CampaignId, how = 'left').drop(clickData.ec_CampaignId)
tempClick2 = tempClick1.join(contactsData, tempClick1.ec_ContactId == contactsData.contactID, how = 'left').drop(contactsData.contactID)

#Agg Clicks
clickagg1 = tempClick2.groupBy('cap_campaignId').agg(#countDistinct('eloq_campaignId').alias('EmailCampaigns'),
                                               count('cap_ContactId').alias('EmailsClicked'),
                                               countDistinct('cap_ContactId').alias('#ClickedContacts'), 
                                               countDistinct('cap_AccountId').alias('#ClickedCompanies'))

#BounceBacks
# #Join Eloqua campaign and Contacts with Email BOUNCES -- ContactIds, CampaignIds, Email Bounces
tempBounce1 = campaignData.join(bounceData, campaignData.eloq_campaignId == bounceData.eb_CampaignId, how = 'left').drop(bounceData.eb_CampaignId)
tempBounce2 = tempBounce1.join(contactsData, tempBounce1.eb_ContactId == contactsData.contactID, how = 'left').drop(contactsData.contactID)

#Agg Sends
bounceagg1 = tempBounce2.groupBy('cap_campaignId').agg(#countDistinct('eloq_campaignId').alias('EmailCampaigns'),
                                                       count('cap_ContactId').alias('EmailsBounced'),
                                                       countDistinct('cap_ContactId').alias('#BouncedContacts'), 
                                                       countDistinct('cap_AccountId').alias('#BouncedCompanies'))


#Join all aggregates
tempagg1 = sendagg1.join(openagg1, sendagg1.cap_campaignId == openagg1.cap_campaignId, how = 'left').drop(openagg1.cap_campaignId)
tempagg2 = tempagg1.join(clickagg1, tempagg1.cap_campaignId == clickagg1.cap_campaignId, how = 'left').drop(clickagg1.cap_campaignId)
tempagg3 = tempagg2.join(bounceagg1, tempagg2.cap_campaignId == bounceagg1.cap_campaignId, how = 'left').drop(bounceagg1.cap_campaignId)

#Filter only valid CampaignIds and ListingIDs
tempagg4 = tempagg3.filter(col('cap_campaignId').startswith('70') | col('cap_campaignId').startswith('a0'))

#Final Summary data
emailData = tempagg4.select(col('cap_campaignId').alias('CampaignId_capforce'), 'EmailCampaigns', 'EmailsSent', '#SentContacts', '#SentCompanies' ,'EmailsOpened', '#OpenedContacts', '#OpenedCompanies', 'EmailsClicked', '#ClickedContacts', '#ClickedCompanies', 'EmailsBounced', '#BouncedContacts','#BouncedCompanies')

In [ ]:
#capData (ListingId)
#boxDownloadData (ListingID)
#adobeData (ad_ListingID)
#emailData(CampaignId_capforce)


listingstemp1 = capData.join(emailData, capData.CampaignId == emailData.CampaignId_capforce, how = 'left').drop(emailData.CampaignId_capforce)
listingstemp2 = listingstemp1.join(adobeData, listingstemp1.ListingId == adobeData.ad_ListingID, how = 'left').drop(adobeData.ad_ListingID)
listingstemp3 = listingstemp2.join(boxDownloadData, listingstemp2.ListingId == boxDownloadData.ListingID, how = 'left').drop(boxDownloadData.ListingID)
listingstemp4 = listingstemp3.join(campaignCapforce, listingstemp3.CampaignId == campaignCapforce.camp_campaignId, how = 'left').drop(campaignCapforce.camp_campaignId)
listingstemp5 = listingstemp4.join(leadBrokerInfo, listingstemp4.OpportunityID == leadBrokerInfo.oppId, how = 'left').drop(leadBrokerInfo.oppId)

finalActiveListings = listingstemp5.select('ListingId', 'ListingName', 'CampaignId', 'OpportunityID', 'CompanyID', 'CompanyName', 'DisplayStatus', col('ListingCreatedDate').cast('Date').alias('ListingCreatedDate'), 'Publish_Date', 'Published_Days', 'AskingPrice', 'DealValue', 'ISOCode', 'CurConvRate','Confidential', 'Property_Type', 'Latitude', 'Longitude', 'State', 'City', 'Zip_Code', 'Status', 'Private_Listing_Name', 'camp_Content_Status', 'camp_IsActive', 'camp_TeamId', 'camp_Status', '#ofExecutedCA', '#ofBidsbyContactsCA', '#ofContactCompaniesCA', '#ofBiddingCompanies', '#PropertyTours', 'MemberExclusion', 'EmailCampaigns', 'EmailsSent', '#SentContacts', '#SentCompanies', 'EmailsBounced', '#BouncedContacts', '#BouncedCompanies', 'EmailsOpened',  '#OpenedContacts', '#OpenedCompanies', 'EmailsClicked', '#ClickedContacts', '#ClickedCompanies', 'TotalVisits', 'PageViews', 'UniqueVisitors', 'TotalBoxDownloads', 'camp_No_of_CampaignMembers', 'camp_NumberOfContacts', 'Primary_Broker_User_Id', 'Primary_Broker_Contact_Id', 'Primary_Broker_Name', 'Primary_Broker_Email', 'Primary_Broker_Phone', 'Secondary_Broker_User_Id', 'Secondary_Broker_Contact_Id', 'Secondary_Broker_Name', 'Secondary_Broker_Email', 'Secondary_Broker_Phone')

In [ ]:
#Adobe Data
adobetemp11.write.mode('overwrite').option('overwriteSchema', 'true').saveAsTable('REMOVED')

#Listings Data
finalActiveListings.write.mode('overwrite').option('overwriteSchema', 'true').saveAsTable('REMOVED')